In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mabdullahsajid/celeba-spoofing")

print("Path to dataset files:", path)

100%|███████████████████████████████████████████████████████████████████████████| 71.9G/71.9G [2:14:24<00:00, 9.58MB/s]

Extracting files...


Path to dataset files: C:\Users\mujta\.cache\kagglehub\datasets\mabdullahsajid\celeba-spoofing\versions\1


In [1]:
import kagglehub

paths = {
    "celeba_spoof": kagglehub.dataset_download("mabdullahsajid/celeba-spoofing"),
    "casia_fasd": kagglehub.dataset_download("immada/casia-fasd"),
    "replay_attack": kagglehub.dataset_download("axondata/replay-attack-mobile-dataset-ibeta-1"),
    "siw": kagglehub.dataset_download("kokomi3012/siw-dataset"),
}

for name, path in paths.items():
    print(f"\n📁 Dataset: {name}")
    print(f"Path: {path}\n")

import os

def list_dir_structure(base_path, max_depth=3, prefix=""):
    if max_depth < 1:
        return
    try:
        for entry in os.listdir(base_path):
            full_path = os.path.join(base_path, entry)
            print(prefix + "├── " + entry)
            if os.path.isdir(full_path):
                list_dir_structure(full_path, max_depth - 1, prefix + "│   ")
    except Exception as e:
        print(f"⚠️ Error reading {base_path}: {e}")

for name, path in paths.items():
    print(f"\n🔍 Directory structure for {name}:\n")
    list_dir_structure(path, max_depth=3)
    print("\n" + "="*80)


📁 Dataset: celeba_spoof
Path: C:\Users\mujta\.cache\kagglehub\datasets\mabdullahsajid\celeba-spoofing\versions\1


📁 Dataset: casia_fasd
Path: C:\Users\mujta\.cache\kagglehub\datasets\immada\casia-fasd\versions\1


📁 Dataset: replay_attack
Path: C:\Users\mujta\.cache\kagglehub\datasets\axondata\replay-attack-mobile-dataset-ibeta-1\versions\4


📁 Dataset: siw
Path: C:\Users\mujta\.cache\kagglehub\datasets\kokomi3012\siw-dataset\versions\1


🔍 Directory structure for celeba_spoof:

├── CelebA_Spoof
│   ├── Data
│   │   ├── test
│   │   ├── train
│   ├── metas
│   │   ├── intra_test
│   │   ├── protocol1
│   │   ├── protocol2
│   ├── README


🔍 Directory structure for casia_fasd:

├── casia-fasd
│   ├── test
│   │   ├── live
│   │   ├── spoof
│   ├── train
│   │   ├── live
│   │   ├── spoof


🔍 Directory structure for replay_attack:

├── Axon Labs Replay mobile sample
│   ├── Galaxy_a54-Honor70
│   │   ├── 20241126_024625.mp4
│   │   ├── 20241126_024646.mp4
│   │   ├── 20241126_024707.mp

In [4]:
import shutil, os

# CASIA-FASD paths
src_train = r"C:\Users\mujta\.cache\kagglehub\datasets\immada\casia-fasd\versions\1\casia-fasd\train"
src_test = r"C:\Users\mujta\.cache\kagglehub\datasets\immada\casia-fasd\versions\1\casia-fasd\test"
dst_root = r"C:\Users\mujta\data"

# Create destination directories
os.makedirs(f"{dst_root}/train/real", exist_ok=True)
os.makedirs(f"{dst_root}/train/spoof", exist_ok=True)
os.makedirs(f"{dst_root}/test/real", exist_ok=True)
os.makedirs(f"{dst_root}/test/spoof", exist_ok=True)

# ✅ Correct mapping: "live" → "real"
shutil.copytree(f"{src_train}/live", f"{dst_root}/train/real", dirs_exist_ok=True)
shutil.copytree(f"{src_train}/spoof", f"{dst_root}/train/spoof", dirs_exist_ok=True)
shutil.copytree(f"{src_test}/live", f"{dst_root}/test/real", dirs_exist_ok=True)
shutil.copytree(f"{src_test}/spoof", f"{dst_root}/test/spoof", dirs_exist_ok=True)

print("✅ CASIA-FASD merged successfully!")

✅ CASIA-FASD merged successfully!


In [5]:
src_siw = r"C:\Users\mujta\.cache\kagglehub\datasets\kokomi3012\siw-dataset\versions\1\SiW"
for split in ["train", "val", "test"]:
    for category in ["real", "spoof"]:
        shutil.copytree(f"{src_siw}/{split}/{category}", f"{dst_root}/{split}/{category}", dirs_exist_ok=True)

In [6]:
import cv2, random

src_replay = r"C:\Users\mujta\.cache\kagglehub\datasets\axondata\replay-attack-mobile-dataset-ibeta-1\versions\4\Axon Labs Replay mobile sample"
dst_spoof = f"{dst_root}/train/spoof"

for folder in os.listdir(src_replay):
    for file in os.listdir(os.path.join(src_replay, folder)):
        if file.endswith(".mp4"):
            video_path = os.path.join(src_replay, folder, file)
            cap = cv2.VideoCapture(video_path)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            for i in random.sample(range(total_frames), min(10, total_frames)):
                cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                ret, frame = cap.read()
                if ret:
                    filename = f"{os.path.splitext(file)[0]}_{i}.jpg"
                    cv2.imwrite(os.path.join(dst_spoof, filename), frame)
            cap.release()

In [5]:
import os, json, shutil
from tqdm import tqdm

# Base paths
base = r"C:\Users\mujta\.cache\kagglehub\datasets\mabdullahsajid\celeba-spoofing\versions\1\CelebA_Spoof"
dst_root = r"C:\Users\mujta\data"

# Create destination folders
for split in ["train", "test"]:
    os.makedirs(f"{dst_root}/{split}/real", exist_ok=True)
    os.makedirs(f"{dst_root}/{split}/spoof", exist_ok=True)

# Load metadata
with open(f"{base}/metas/protocol1/train_label.json", "r") as f:
    meta_train = json.load(f)

with open(f"{base}/metas/protocol1/test_label.json", "r") as f:
    meta_test = json.load(f)


def copy_split(meta, split):
    for img_rel_path, label_info in tqdm(meta.items(), total=len(meta), desc=f"Copying {split}"):
        # Determine full source path
        src = os.path.join(base, img_rel_path)

        # CelebA Spoof labels are stored as lists, where last element is label
        # Example: [0, 1, 0, 1] -> spoof label is last (label_info[-1])
        if isinstance(label_info, list):
            spoof_label = label_info[-1]
        elif isinstance(label_info, dict) and "spoof_label" in label_info:
            spoof_label = label_info["spoof_label"]
        else:
            continue  # unknown format, skip

        # Destination folder
        dst = f"{dst_root}/{split}/spoof" if spoof_label == 1 else f"{dst_root}/{split}/real"

        try:
            shutil.copy(src, dst)
        except FileNotFoundError:
            continue


# Run for both splits
copy_split(meta_train, "train")
copy_split(meta_test, "test")

print("✅ CelebA-Spoof dataset merged successfully!")

Copying test: 100%|██████████████████████████████████████████████████████████████| 25758/25758 [07:45<00:00, 55.36it/s]

✅ CelebA-Spoof dataset merged successfully!


In [12]:
dst_root = r"C:\Users\mujta\data"

for split in ["train", "val", "test"]:
    for cls in ["real", "spoof"]:
        count = len(os.listdir(f"{dst_root}/{split}/{cls}"))
        print(f"{split}/{cls}: {count}")

train/real: 106614
train/spoof: 201883
val/real: 600
val/spoof: 150
test/real: 19639
test/spoof: 72655


In [13]:
import os, random, shutil
from tqdm import tqdm

def balance_dataset(root):
    for split in ["train", "val", "test"]:
        real_dir = os.path.join(root, split, "real")
        spoof_dir = os.path.join(root, split, "spoof")

        real_imgs = os.listdir(real_dir)
        spoof_imgs = os.listdir(spoof_dir)

        n_real = len(real_imgs)
        n_spoof = len(spoof_imgs)

        # If spoof > real → reduce spoof
        if n_spoof > n_real:
            spoof_imgs = random.sample(spoof_imgs, n_real)
        # If real > spoof → reduce real
        elif n_real > n_spoof:
            real_imgs = random.sample(real_imgs, n_spoof)

        # Move balanced images into new folder
        balanced_root = root + "_balanced"
        os.makedirs(os.path.join(balanced_root, split, "real"), exist_ok=True)
        os.makedirs(os.path.join(balanced_root, split, "spoof"), exist_ok=True)

        for img in tqdm(real_imgs, desc=f"{split}/real"):
            shutil.copy(os.path.join(real_dir, img), os.path.join(balanced_root, split, "real", img))
        for img in tqdm(spoof_imgs, desc=f"{split}/spoof"):
            shutil.copy(os.path.join(spoof_dir, img), os.path.join(balanced_root, split, "spoof", img))

    print("✅ Dataset balanced and saved to:", balanced_root)

balance_dataset(dst_root)

test/spoof: 100%|████████████████████████████████████████████████████████████████| 19639/19639 [04:23<00:00, 74.66it/s]

✅ Dataset balanced and saved to: C:\Users\mujta\data_balanced


In [14]:
import os
import cv2
import torch
from facenet_pytorch import MTCNN
from tqdm import tqdm
from PIL import Image
import numpy as np

# Paths
src_root = r"C:\Users\mujta\data_balanced"
dst_root = r"C:\Users\mujta\data_faces"

# Initialize MTCNN (GPU if available)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=224, margin=20, keep_all=False, post_process=False, device=device)

# Create destination dirs
for split in ["train", "val", "test"]:
    for label in ["real", "spoof"]:
        os.makedirs(f"{dst_root}/{split}/{label}", exist_ok=True)


def crop_faces(split):
    for label in ["real", "spoof"]:
        src_dir = f"{src_root}/{split}/{label}"
        dst_dir = f"{dst_root}/{split}/{label}"

        for img_name in tqdm(os.listdir(src_dir), desc=f"{split}/{label}"):
            src_path = os.path.join(src_dir, img_name)
            dst_path = os.path.join(dst_dir, img_name)
            try:
                img = cv2.imread(src_path)
                if img is None:
                    continue

                # Convert BGR → RGB for MTCNN
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_pil = Image.fromarray(img_rgb)

                # Detect and crop
                face = mtcnn(img_pil)

                if face is not None:
                    # Convert tensor → NumPy image
                    face = face.permute(1, 2, 0).byte().cpu().numpy()
                    face_bgr = cv2.cvtColor(face, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(dst_path, face_bgr)

            except Exception as e:
                continue


# Process splits
crop_faces("train")
crop_faces("val")
crop_faces("test")

print("✅ Faces cropped and aligned successfully!")

test/spoof: 100%|████████████████████████████████████████████████████████████████| 19639/19639 [25:25<00:00, 12.87it/s]

✅ Faces cropped and aligned successfully!


In [15]:
for split in ["train", "val", "test"]:
    for cls in ["real", "spoof"]:
        count = len(os.listdir(f"{dst_root}/{split}/{cls}"))
        print(f"{split}/{cls}: {count}")

train/real: 106406
train/spoof: 103599
val/real: 150
val/spoof: 150
test/real: 19618
test/spoof: 19393


In [ ]:
# ================================================
# FACE LIVENESS DETECTION (REAL vs SPOOF)
# Using ViT-B/16 + Focal Loss + Data Augmentation
# ================================================

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import transforms, datasets, models
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np
from tqdm import tqdm

# ================================================
# 1. CONFIG
# ================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

data_dir = "data_faces"  # must contain subfolders: real/, spoof/
batch_size = 32
num_epochs = 10
lr = 1e-4
save_path = "best_vit_liveness.pth"

# ================================================
# 2. TRANSFORMS
# ================================================
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# ================================================
# 3. DATASETS
# ================================================
train_dir = os.path.join(data_dir, "train")
val_dir   = os.path.join(data_dir, "val")

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset   = datasets.ImageFolder(val_dir, transform=val_transform)

print("Class to idx:", train_dataset.class_to_idx)

# ================================================
# 4. BALANCED SAMPLER
# ================================================
targets = [label for _, label in train_dataset.samples]
class_count = np.bincount(targets)
weights = 1. / torch.tensor(class_count, dtype=torch.float)
sample_weights = [weights[t] for t in targets]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ================================================
# 5. MODEL (ViT-B/16)
# ================================================
from torchvision.models import vit_b_16, ViT_B_16_Weights

weights = ViT_B_16_Weights.IMAGENET1K_V1
model = vit_b_16(weights=weights)
model.heads.head = nn.Linear(model.heads.head.in_features, 2)
model = model.to(device)

# ================================================
# 6. FOCAL LOSS
# ================================================
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    def forward(self, inputs, targets):
        CE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-CE_loss)
        return (self.alpha * (1-pt)**self.gamma * CE_loss).mean()

criterion = FocalLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
best_val_loss = float('inf')

# ================================================
# 7. TRAINING LOOP
# ================================================
def calculate_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    acc = (tp + tn) / (tp + tn + fp + fn)
    apcer = fp / (tn + fp + 1e-6)
    bpcer = fn / (tp + fn + 1e-6)
    acer = (apcer + bpcer) / 2
    f1 = f1_score(y_true, y_pred)
    return acc, f1, apcer, acer

for epoch in range(1, num_epochs+1):
    model.train()
    train_loss, preds, labels = 0, [], []
    for imgs, lbls in tqdm(train_loader, desc=f"Epoch {epoch} Train"):
        imgs, lbls = imgs.to(device), lbls.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, lbls)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * imgs.size(0)
        preds.extend(torch.argmax(outputs, 1).cpu().numpy())
        labels.extend(lbls.cpu().numpy())
    train_loss /= len(train_dataset)
    train_acc, train_f1, train_apcer, train_acer = calculate_metrics(labels, preds)

    # Validation
    model.eval()
    val_loss, preds, labels = 0, [], []
    with torch.no_grad():
        for imgs, lbls in tqdm(val_loader, desc=f"Epoch {epoch} Val"):
            imgs, lbls = imgs.to(device), lbls.to(device)
            outputs = model(imgs)
            loss = F.cross_entropy(outputs, lbls)
            val_loss += loss.item() * imgs.size(0)
            preds.extend(torch.argmax(outputs, 1).cpu().numpy())
            labels.extend(lbls.cpu().numpy())
    val_loss /= len(val_dataset)
    val_acc, val_f1, val_apcer, val_acer = calculate_metrics(labels, preds)

    print(f"\nEpoch {epoch} | LR: {lr}")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    print(f"Train ACC/F1/ACER: {train_acc:.4f}/{train_f1:.4f}/{train_acer:.4f}")
    print(f"Val   ACC/F1/ACER: {val_acc:.4f}/{val_f1:.4f}/{val_acer:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), save_path)
        print("✅ Saved best model.\n")
    else:
        print("No improvement.\n")

print("Training complete.")

Class to idx: {'real': 0, 'spoof': 1}


Epoch 1 Val: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.06s/it]



Epoch 1 | Time: 21133.4s | LR: 0.000500
Train Loss: 0.4733 | Val Loss: 1.2170
Train ACC/F1/ACER: 0.8194 / 0.8170 / 0.1806
Val   ACC/F1/ACER: 0.5400 / 0.1481 / 0.4600
✅ Saved best model.


Epoch 2 Train:   7%|████                                                          | 216/3282 [21:31<5:00:05,  5.87s/it]

In [ ]:
# -----------------------------------------------------

# ---------------- Final Test Eval --------------------
model.eval()
test_preds, test_labels = [], []
with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        logits = model(imgs)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        test_preds.append(preds)
        test_labels.append(labels.cpu().numpy())

test_preds, test_labels = np.concatenate(test_preds), np.concatenate(test_labels)
test_stats = compute_metrics(test_labels, test_preds)

print("\n--- Test Metrics ---")
print("Confusion Matrix:\n", test_stats["cm"])
print(f"Accuracy:  {test_stats['acc']:.4f}")
print(f"Precision: {test_stats['prec']:.4f}")
print(f"Recall:    {test_stats['rec']:.4f}")
print(f"F1 Score:  {test_stats['f1']:.4f}")
print(f"APCER:     {test_stats['APCER']:.4f}")
print(f"BPCER:     {test_stats['BPCER']:.4f}")
print(f"ACER:      {test_stats['ACER']:.4f}")

torch.save(model.state_dict(), "final_model_weights.pth")
print("💾 Saved final_model_weights.pth")